In [6]:
"""
Stuff Documents 체인을 사용하여 완전한 RAG 파이프라인을 구현하세요.
체인을 수동으로 구현해야 합니다.
체인에 ConversationBufferMemory를 부여합니다.
체인에 다음 질문을 합니다:
- Is Aaronson guilty?
- What message did he write in the table?
- Who is Julia?
"""

# Retreiver
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import Chroma
from langchain.storage import LocalFileStore

# Memory
from langchain.memory import ConversationBufferMemory

# Chat
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough

chat = ChatOpenAI(
temperature=0.1,
)

In [7]:
loader = TextLoader("/Users/jihoonchoi/Projects/GPTs/Challenge/Challenge_files/document.txt")
	
splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n", chunk_size=600, chunk_overlap=100,
    )

docs = loader.load_and_split(text_splitter=splitter) 
	
embeddings = OpenAIEmbeddings() 

cache_dir = LocalFileStore("/Users/jihoonchoi/Projects/GPTs/Challenge/Challenge_cache/embeddings") 
cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

vectorstore = Chroma.from_documents(docs, cached_embeddings)
retriever = vectorstore.as_retriever()

In [8]:
memory = ConversationBufferMemory(
	llm = chat,
	max_token_limit = 200,
	memory_key='chat_history',
	return_messages = True,
	)

def load_memory():
	return memory.load_memory_variables({})["chat_history"]

In [9]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            Answer the question using ONLY the following context. If you don't know the answer just say you don't know. DON'T make anything up.
            
            Context: {context}
            """,
        ),
        ("human", "{question}"),
    ]
)

llm = chat

chain = {"context": retriever, "question": RunnablePassthrough()} | prompt | llm

chain.invoke("Who is Julia?")

AIMessage(content='Julia is a character mentioned in the context.')